Could the non-ML pYIN algorithm be faster than CREPE?
Turns out it isn't!

In [ ]:
from ddsp.spectral_ops import compute_f0
from thesis.notebook_util import specplot, play_audio
import librosa

In [ ]:
import ddsp.training.data
data_provider = ddsp.training.data.TFRecordProvider(
    "../data/datasets/violin3/violin3.tfrecord*",
    frame_rate=50,
    centered=True,
    with_jukebox=True,
)
dataset = data_provider.get_batch(batch_size=1, shuffle=True, repeats=1)
dataset_iter = iter(dataset)

In [ ]:
batch = next(dataset_iter)
audio = batch["audio_16k"]
play_audio(audio)

In [ ]:
%%timeit
f0_hz_crepe, f0_confidence_crepe = compute_f0(audio[0], frame_rate=50, model_name="crepe-full")

In [ ]:
%%timeit
f0_hz_pyin, _, f0_confidence_pyin = librosa.pyin(
    audio[0].numpy(),
    fmin=librosa.note_to_hz('C2'),
    fmax=librosa.note_to_hz('C7'),
    sr=16000,
    frame_length=(16000//50)*4,
)

In [ ]:
assert f0_hz_pyin.shape == f0_hz_crepe.shape

In [ ]:
import matplotlib.pyplot as plt

plt.plot(f0_hz_crepe)
plt.plot(f0_hz_pyin)

In [ ]:
specplot(audio[0])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(f0_confidence_crepe)
plt.plot(f0_confidence_pyin)